In [9]:
!apt install libopenblas-base libomp-dev
!pip install faiss
import numpy as np 
import faiss 
import json

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-base is already the newest version (0.2.20+ds-4).
libopenblas-base set to manually installed.
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 29 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 2s (130 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 160975 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1

In [52]:
from google.colab import drive
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
drive.mount('/content/drive')
model = api.load('glove-twitter-50')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
threshold = 0.75
N = 1
with open('/content/drive/My Drive/Augment4Gains/data/reddit/train.json') as f:
  data = json.load(f)
  for each_data in data:
    sentence = each_data['source']
    print(sentence)
    #processed_sentence = re.sub('[^a-zA-Z]', ' ', processed_sentence )
    processed_sentence = re.sub(r'\s+', ' ', sentence)
    words = word_tokenize(processed_sentence)
    #pos_tagging goes here
    word_pos = nltk.pos_tag(words)
    for idx, word in enumerate(words):
      if word in model.vocab:
        if word_pos[idx][1] in ['VB','NN','JJ']:
          #get words with highest cosine similarity
          replacements = model.most_similar(positive=word, topn=N)
          #keep only words that pass the threshold
          replacements = [replacements[i][0] for i in range(N) if replacements[i][1] > threshold]
          #check for POS tag equality, dismiss if unequal
          replacements = [elem for elem in replacements if nltk.pos_tag([elem.lower()])[0][1] == word_pos[idx][1]]
          if replacements:
            words[idx] = replacements[0]
    new_sentence = TreebankWordDetokenizer().detokenize(words)
    print(new_sentence)
    print('')


if you give me that salsa i will love you forever . spicy is the shit .
if you give me that merengue i will love you forever . spicy is the damn.
